## Dependencies required are pyaudio, speechrecognition, playsound, gTTS

In [1]:
import speech_recognition as sr
import os, pyaudio, wave, time, csv
from gtts import gTTS
from playsound import playsound
from threading import Thread
recognizer=sr.Recognizer()

In [60]:
#generate dummy questions and probable answers
questions = ["के तपाईको टाउको दुख्छ", "के तपाइलाई ज्वरो छ ", "के तपाई को नाक बग्छ ", "के तपाइलाई सास फेर्न समस्या छ "]
for i in range(0, len(questions)):
    speech = gTTS(text[i], lang = "en-IN", slow = False)
    name="questions/"+str(i)+".mp3"
    speech.save(name)
    
ans=[
     ["छ", "छैन","दुख्छ", "दुख्दैन","अलिअलि दुख्छ","धेरै दुख्छ", "दुखिरख्छ", "दुखेको छ", "दुखेको छैन"],
     ["छ", "छैन", "अलिअलि छ", "धेरै छ", "आइरको छ", "आको छैन"],
     ["छ", "छैन", "बग्छ", "बग्दैन", "बगेको छ", "बगेको छैन" ],
     ["छ", "छैन", "समस्या छ", "समस्या छैन", "अलिअलि छ"]
]

In [5]:
def recognize(filename, lang="ne-NP"):
    my_file=sr.AudioFile(filename)
    with my_file as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.record(source)
    try:
        output=recognizer.recognize_google(audio, language=lang, show_all=True)
    except sr.RequestError:
        output="API unavailable or did not response"
    except sr.UnknownValueError:
        output="Did not recognize the audio"
    return output

In [77]:
def record_and_save(num):
    time.sleep(1)
    print("go")
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 2
    fs = 44100
    seconds = 5
    filename = "recorded/"+str(phone_num)+"_"+str(num)+".wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []


    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)


    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()

In [83]:
#record here
from random import randint
phone_num=randint(9800000000,9869999999)
for i in range(0, 4):
    temp="questions/"+str(i)+".mp3"
    
    #thread creation
    t1=Thread(target=playsound, args=(temp,))
    t2=Thread(target=record_and_save, args=(i,))

    t1.start()
    t2.start()

    t1.join()
    t2.join()

go
go
go
go


In [84]:
#recognize here
output=[]
for i in range(0, 4):
    temp=[]
    filename="recorded/"+str(phone_num)+"_"+str(i)+".wav"
    converted=recognize(filename)
    if(len(converted)==0):
        output.append("err")
    else:
        for a in converted['alternative']:
            temp.append(a['transcript'])
        output.append(temp)

        
        
#append to csv here
import numpy as np
results=[phone_num]
for i in range(0,4):
    c = [value for value in ans[i] if value in output[i]]
    if output[i]=="err":
        results.append("err")
    else:
        if len(c)>0:
            results.append(c[0])
        else:
            results.append("*")
        
with open(r'results.csv', 'a', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(results)

print(output)
print()
print(results)

In [87]:
import pandas as pd
pd.read_csv("results.csv")

,phone_num,1,2,3,4
0,9809200114,दुख्छ,छैन,बग्छ,छैन
1,9809571247,दुखेको छ,*,बग्दैन,छ


In [ ]:
def listen_and_recognize(lang="ne-NP"):
    mic = sr.Microphone(device_index=1)
    with mic as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    try:
        output=recognizer.recognize_google(audio, language=lang)
    except sr.RequestError:
        output="API unavailable"
    except sr.UnknownValueError:
        output="recognition err"
    return output

def listen_and_save(num, lang="ne-NP"):
    mic = sr.Microphone(device_index=1)
    with mic as source:
        audio = recognizer.listen(source)
        filename="recorded/rec_"+str(num)+".wav"
        with open(filename, "wb") as f:
            f.write(audio.get_wav_data())

#listen and recognize here
for i in range(0, 4):
    temp="questions/text_"+str(i)+".mp3"
    playsound(temp)
    print(listen_and_recognize(i))
    
#listen and save here
for i in range(0, 4):
    temp="questions/text_"+str(i)+".mp3"
    playsound(temp)
    print(listen_and_save(i))